In [1]:
import os
import re
import io
import nltk
import string
import math
from pycontractions import Contractions

In [2]:
os.chdir('C:/Users/Vedant/Desktop/NLP LAB/N-gram dataset/')

In [3]:
pat = 'C:/Users/Vedant/Desktop/NLP LAB/N-gram dataset/'

In [4]:
Data = []

In [5]:
for y in range(4,7):
    path = os.path.join(pat,'200'+ str(y))
    os.chdir(path)
    for filename in os.listdir():
        f=0
        with io.open(filename,'r') as f:
            file = f.read()
        if 'index' not in filename:
            for st in re.split('\n',file):
                if '200'+str(y) in st:
                    f=1
                    continue
                if not (st == '' or st[0] == '<'):
                    if f == 1:
                        Data.append(st)
        else:
            for st in re.split('\n',file):
                if '1999' in st:
                    f=1
                    continue
                if not (st == '' or st[0] == '<'):
                    if f == 1:
                        Data.append(st)

In [6]:
test = []

In [7]:
y = 7
path = os.path.join(pat,'200'+ str(y))
os.chdir(path)
for filename in os.listdir():
    f=0
    with io.open(filename,'r') as f:
        file = f.read()
    if 'index' not in filename:
        for st in re.split('\n',file):
            if '200'+str(y) in st:
                f=1
                continue
            if not (st == '' or st[0] == '<'):
                if f == 1:
                    test.append(st)
    else:
        for st in re.split('\n',file):
            if '1999' in st:
                f=1
                continue
            if not (st == '' or st[0] == '<'):
                if f == 1:
                    test.append(st)

In [8]:
UNK = None

In [9]:
class UnigramLanguageModel:
    def __init__(self, sentences, smoothing=False):
        self.unigram_frequencies = dict()
        self.corpus_length = 0
        for sentence in sentences:
            for word in sentence:
                self.unigram_frequencies[word] = self.unigram_frequencies.get(word, 0) + 1
                self.corpus_length += 1
        self.unique_words = len(self.unigram_frequencies)
        self.smoothing = smoothing

    def calculate_unigram_probability(self, word):
            word_probability_numerator = self.unigram_frequencies.get(word, 0)
            word_probability_denominator = self.corpus_length
            if self.smoothing:
                word_probability_numerator += 1
                word_probability_denominator += self.unique_words + 1
            return float(word_probability_numerator) / float(word_probability_denominator)

    def calculate_sentence_probability(self, sentence, normalize_probability=True):
        sentence_probability_log_sum = 0
        for word in sentence:
            word_probability = self.calculate_unigram_probability(word)
            sentence_probability_log_sum += math.log(word_probability, 2)
        return math.pow(2, sentence_probability_log_sum) if normalize_probability else sentence_probability_log_sum                

    def sorted_vocabulary(self):
        full_vocab = list(self.unigram_frequencies.keys())
        full_vocab.sort()
        full_vocab.append(UNK)
        return full_vocab
    
class BigramLanguageModel(UnigramLanguageModel):
    def __init__(self, sentences, smoothing=False):
        UnigramLanguageModel.__init__(self, sentences, smoothing)
        self.bigram_frequencies = dict()
        self.unique_bigrams = set()
        for sentence in sentences:
            previous_word = None
            for word in sentence:
                if previous_word != None:
                    self.bigram_frequencies[(previous_word, word)] = self.bigram_frequencies.get((previous_word, word),0) + 1
                    self.unique_bigrams.add((previous_word, word))
                previous_word = word
        self.unique__bigram_words = len(self.unigram_frequencies)

    def calculate_bigram_probabilty(self, previous_word, word):
        bigram_word_probability_numerator = self.bigram_frequencies.get((previous_word, word), 0)
        bigram_word_probability_denominator = self.unigram_frequencies.get(previous_word, 0)
        if self.smoothing:
            bigram_word_probability_numerator += 1
            bigram_word_probability_denominator += self.unique__bigram_words
        if bigram_word_probability_numerator == 0 or bigram_word_probability_denominator == 0:
            return 0.0
        else:
            return float(bigram_word_probability_numerator) / float(bigram_word_probability_denominator)

    def calculate_bigram_sentence_probability(self, sentence, normalize_probability=True):
        bigram_sentence_probability_log_sum = 0
        previous_word = None
        for word in sentence:
            if previous_word != None:
                bigram_word_probability = self.calculate_bigram_probabilty(previous_word, word)
                bigram_sentence_probability_log_sum += math.log(bigram_word_probability, 2)
            previous_word = word
        return math.pow(2,bigram_sentence_probability_log_sum) if normalize_probability else bigram_sentence_probability_log_sum

class TrigramLanguageModel(BigramLanguageModel):
    def __init__(self,sentences,smoothing = False):
        BigramLanguageModel.__init__(self, sentences, smoothing)
        self.trigram_frequencies = dict()
        self.unique_trigrams = set()
        for sentence in sentences:
            first_word = None
            second_word = None
            for word in sentence:
                if second_word != None and first_word != None:
                    self.trigram_frequencies[(second_word,first_word, word)] = self.trigram_frequencies.get((second_word,first_word, word),0) + 1
                    self.unique_trigrams.add((second_word,first_word, word))
                second_word = first_word
                first_word = word
        self.unique__trigram_words = len(self.unigram_frequencies)
    def calculate_trigram_probabilty(self,second_word,first_word,word):
        trigram_word_probability_numerator = self.trigram_frequencies.get((second_word,first_word, word), 0)
        trigram_word_probability_denominator = self.bigram_frequencies.get((second_word,first_word), 0)
        if self.smoothing:
            trigram_word_probability_numerator += 1
            trigram_word_probability_denominator += self.unique__trigram_words
        return 0.0 if trigram_word_probability_numerator == 0 or trigram_word_probability_denominator == 0 else float(
            trigram_word_probability_numerator) / float(trigram_word_probability_denominator)

    def calculate_trigram_sentence_probability(self, sentence, normalize_probability=True):
        trigram_sentence_probability_log_sum = 0
        second_word = None
        first_word = None
        
        for word in sentence:
            if second_word != None and first_word != None:
                trigram_word_probability = self.calculate_trigram_probabilty(second_word,first_word, word)
                trigram_sentence_probability_log_sum += math.log(trigram_word_probability, 2)
            first_word = word
            second_word = first_word
        return math.pow(2,
                        trigram_sentence_probability_log_sum) if normalize_probability else trigram_sentence_probability_log_sum

    
def calculate_number_of_unigrams(sentences):
    unigram_count = 0
    for sentence in sentences:
        unigram_count += len(sentence)
    return unigram_count
def calculate_number_of_bigrams(sentences):
        bigram_count = 0
        for sentence in sentences:
            bigram_count += len(sentence) - 1
        return bigram_count
def calculate_number_of_trigrams(sentences):
        trigram_count = 0
        for sentence in sentences:
            trigram_count += len(sentence) - 2
        return trigram_count
def calculate_unigram_perplexity(model, sentences):
    unigram_count = calculate_number_of_unigrams(sentences)
    sentence_probability_log_sum = 0
    for sentence in sentences:
        try:
            sentence_probability_log_sum -= math.log(model.calculate_sentence_probability(sentence), 2)
        except:
            pass
    return math.pow(2, sentence_probability_log_sum / unigram_count)

def calculate_bigram_perplexity(model, sentences):
    number_of_bigrams = calculate_number_of_bigrams(sentences)
    bigram_sentence_probability_log_sum = 0
    for sentence in sentences:
        try:
            bigram_sentence_probability_log_sum -= math.log(model.calculate_bigram_sentence_probability(sentence), 2)
        except:
            pass
    return math.pow(2, bigram_sentence_probability_log_sum / number_of_bigrams)

def calculate_trigram_perplexity(model, sentences):
    number_of_trigrams = calculate_number_of_trigrams(sentences)
    trigram_sentence_probability_log_sum = 0
    for sentence in sentences:
        try:
            trigram_sentence_probability_log_sum -= math.log(model.calculate_trigram_sentence_probability(sentence), 2)
        except:
            pass
    return math.pow(2, trigram_sentence_probability_log_sum / number_of_trigrams)

In [10]:
from string import digits
remove_digits = str.maketrans('', '', digits)
from string import punctuation
table = str.maketrans('','',punctuation)
cont = Contractions(api_key="glove-twitter-100")

In [11]:
cont.load_models()

In [12]:
for d in range(len(Data)):
    s = Data[d]
    s = s.lower()
    s = s.translate(table)
    s = s.translate(remove_digits)
    s = list(cont.expand_texts([s]))[0]
    s = nltk.word_tokenize(s)
    Data[d] = s

In [13]:
dataset_model = TrigramLanguageModel(Data)

In [14]:
dataset_model_smoothed = TrigramLanguageModel(Data,smoothing = True)

In [15]:
sorted_vocab_keys = dataset_model.sorted_vocabulary()

In [16]:
print("== TRAIN PERPLEXITY == ")
print("unigram: ", calculate_unigram_perplexity(dataset_model, Data))
print("bigram: ", calculate_bigram_perplexity(dataset_model, Data))
print("trigram: ", calculate_trigram_perplexity(dataset_model, Data))

== TRAIN PERPLEXITY == 
unigram:  1535.4401039431868
bigram:  82.7156875550523
trigram:  1.0000052122827534


In [17]:
print("== TRAIN PERPLEXITY == ")
print("unigram: ", calculate_unigram_perplexity(dataset_model_smoothed, Data))
print("bigram: ", calculate_bigram_perplexity(dataset_model_smoothed, Data))
print("trigram: ", calculate_trigram_perplexity(dataset_model_smoothed, Data))

== TRAIN PERPLEXITY == 
unigram:  1545.508219559923
bigram:  3416.6267911858044
trigram:  40033.6378309513


In [18]:
for d in range(len(test)):
    s = test[d]
    s = s.lower()
    s = s.translate(table)
    s = s.translate(remove_digits)
    s = nltk.word_tokenize(s)
    test[d] = s

In [19]:
print("== TEST PERPLEXITY == ")
print("unigram: ", calculate_unigram_perplexity(dataset_model, test))
print("bigram: ", calculate_bigram_perplexity(dataset_model, test))
print("trigram: ", calculate_trigram_perplexity(dataset_model, test))

== TEST PERPLEXITY == 
unigram:  98.70888683427059
bigram:  1.0437204499672985
trigram:  1.0000786856105486


In [20]:
print("== TEST PERPLEXITY == ")
print("unigram: ", calculate_unigram_perplexity(dataset_model_smoothed, test))
print("bigram: ", calculate_bigram_perplexity(dataset_model_smoothed, test))
print("trigram: ", calculate_trigram_perplexity(dataset_model_smoothed, test))

== TEST PERPLEXITY == 
unigram:  1920.9823811109472
bigram:  5247.631636500524
trigram:  72673.97032516371


Trigram Language Model

In [34]:
dataset_model_smoothed = TrigramLanguageModel(test,smoothing = True)

In [35]:
sorted_vocab_keys = dataset_model_smoothed.sorted_vocabulary()

In [36]:
len(sorted_vocab_keys)

8245

In [38]:
Least_probability = 1.0/float(dataset_model_smoothed.unique__bigram_words)

Sentence Generation : Trigram

In [83]:
sent = {}
for x in dataset_model_smoothed.trigram_frequencies.keys():
    if (x[0],x[1]) not in sent.keys():
        sent[(x[0],x[1])] = {}
    sent[(x[0],x[1])][x[2]] = float(dataset_model_smoothed.trigram_frequencies[(x[0],x[1],x[2])])/float(dataset_model_smoothed.bigram_frequencies[(x[0],x[1])])
    

In [84]:
for x in sent.keys():
    sent[x] = dict(sorted(sent[x].items(),key=lambda x: x[1]))

In [125]:
import random
sentence_finished = False
text = ['today',
 'the']
while not sentence_finished:
    r = random.random()
    acc = .0
    for word in sent[tuple(text[-2:])].keys():
        acc += sent[tuple(text[-2:])][word]
        if acc >= r:
            text.append(word)
            break

    if text[-2:] == ['None', 'None'] or len(text) > 30:
        sentence_finished = True
    if tuple(text[-2:]) not in sent.keys():
        sent[tuple(text[-2:])] = {'None':Least_probability}
        
print (' '.join([t for t in text if t]))

today the latest conviction in an attack that renewed longrun None None


Sentence Generation : Bigram

In [ ]:
sent1 = {}
for x in dataset_model_smoothed.bigram_frequencies.keys():
    if x[0] not in sent1.keys():
        sent1[x[0]] = {}
    sent1[x[0]][x[1]] = float(dataset_model_smoothed.bigram_frequencies[(x[0],x[1])])/float(dataset_model_smoothed.unigram_frequencies[x[0]])

In [97]:
for x in sent1.keys():
    sent1[x] = dict(sorted(sent1[x].items(),key=lambda x: x[1]))

In [126]:
import random
sentence_finished = False
text = ['today', 'the']
while not sentence_finished:
    r = random.random()
    acc = .0
    if text[-1] not in sent1.keys():
        break
    for word in sent1[text[-1]].keys():
        acc += sent1[text[-1]][word]
        if acc >= r:
            text.append(word)
            break

    if text[-1] == 'None' or len(text) > 30:
        sentence_finished = True
    if text[-1] not in sent.keys():
        sent1[text[-1]] = {'None':Least_probability}
        
print (' '.join([t for t in text if t]))

today the us None
